In [2]:
import MeCab

# Define a function that extracts corpus from a sentence
def extract_corpus(text):
    titles = []
    m = MeCab.Tagger('-d /jupyter/mecab-ko-dic-2.1.1-20180720')
    parsedtext = m.parse(text)
    parsedtext_line = parsedtext.split('\n')
    for syl in parsedtext_line:
        if(syl.count(',')==7):
            index = syl.find(',')
            index = syl.replace(',', '-', 1).find(',')
            text = (syl[0:index].replace("\t",","))
            textinbit = (''.join(format(ord(x), 'b') for x in syl))
            #print(text)
        elif(syl.count(',')==8):
            index = syl.find(',')
            index = syl.replace(',', '-', 1).replace(',', '-', 1).find(',')
            text = (syl[0:index].replace("\t",","))
            textinbit = (''.join(format(ord(x), 'b') for x in syl))
            #print(text)
        titles.append(text)  
    return(titles)

In [3]:
# Load File
lines = []
f = open('cqrs_shared.txt','rt', encoding='utf-8')  # Open file with 'UTF-8' 인코딩
while True:
    line = f.readline()
    lines.append(line)
    if not line:
        break
f.close()

# Use part if original text data is too large
lines = lines[0:10]

titles = []
remove_characters = []
for title in lines:
    for remove_character in remove_characters:
        title = title.replace(remove_character,"")
    titles.append(title)

# Array of Corpus List for Each Sentence
corpus = []
    
for title in titles:
    corpus.append(extract_corpus(title))

In [4]:
words = []
for text in corpus:
    for word in text:
        words.append(word)

words = set(words)
#print(words)
print(len(words))

151


In [5]:
word2int = {}

for i,word in enumerate(words):
    word2int[word] = i
    #word2int[word] = (''.join(format(ord(x), 'b') for x in word))

sentences = []
for sentence in corpus:
    sentences.append(sentence)
    
WINDOW_SIZE = 2

data = []
for sentence in sentences:
    for idx, word in enumerate(sentence):
        for neighbor in sentence[max(idx - WINDOW_SIZE, 0) : min(idx + WINDOW_SIZE, len(sentence)) + 1] : 
            if neighbor != word:
                data.append([word, neighbor])

In [6]:
import pandas as pd
df = pd.DataFrame(data, columns = ['input', 'label'])

In [7]:
df.head(10)

,input,label
0,"외신,NNG,*","충격,NNG,*"
1,"외신,NNG,*","보도,NNG,*"
2,"충격,NNG,*","외신,NNG,*"
3,"충격,NNG,*","보도,NNG,*"
4,"충격,NNG,*","“,SSO,*"
5,"보도,NNG,*","외신,NNG,*"
6,"보도,NNG,*","충격,NNG,*"
7,"보도,NNG,*","“,SSO,*"
8,"보도,NNG,*","文,NNG,*"
9,"“,SSO,*","충격,NNG,*"


In [8]:
df.shape

(880, 2)

In [9]:
word2int

{'에,JKB,*': 0,
 '’,SY,*': 1,
 '축,NNBC,*': 2,
 '사용,NNG,행위': 3,
 '요건,NNG,*': 4,
 '뇌물,NNG,*': 5,
 '삭감,NNG,행위': 6,
 '자료,NNG,*': 7,
 '과,JKB,*': 8,
 '헌법,NNG,*': 9,
 '회담,NNG,*': 10,
 '맹비난,NNG,*': 11,
 '국제,NNG,*': 12,
 '”,SSC,*': 13,
 '文,NNG,*': 14,
 '‘,SY,*': 15,
 '문건,NNG,*': 16,
 '이나,JC,*': 17,
 '정부,NNG,*': 18,
 '김문수,NNP,인명': 19,
 '박근혜,NNP,인명': 20,
 '행위,NNG,행위': 21,
 '국방부,NNG,*': 22,
 '법,NNG,*': 23,
 '장관,NNG,*': 24,
 '는,ETM,*': 25,
 '포괄,NNG,*': 26,
 '며,EC,*': 27,
 '영화제,NNG,*': 28,
 '의원,NNG,*': 29,
 '김정은,NNP,인명': 30,
 '고,EC,*': 31,
 '무시,NNG,행위': 32,
 '“,SSO,*': 33,
 '으로,JKB,*': 34,
 '불복종,NNG,*': 35,
 '외침,NNG,*': 36,
 '비,XPN,*': 37,
 '누가,NP+JKS,*': 38,
 '美,NNG,*': 39,
 '님,XSN,*': 40,
 '재판,NNG,*': 41,
 '위수령,NNG,*': 42,
 '수수,NNG,*': 43,
 '자유,NNG,정적사태': 44,
 '대통령,NNG,*': 45,
 '내,NP,*': 46,
 '은,JX,*': 47,
 '부리,VV,*': 48,
 '여론,NNP,*': 49,
 '상영,NNG,행위': 50,
 '반영,NNG,*': 51,
 '공개,NNG,행위': 52,
 '충격,NNG,*': 53,
 '나,EC,*': 54,
 '전,MM,~명사': 55,
 '국가,NNG,*': 56,
 '취소,NNG,행위': 57,
 '교과서,NNG,*': 58,
 '운운,NN

In [30]:
import tensorflow as tf
import numpy as np

ONE_HOT_DIM = len(words)

# function to convert numbers to one hot vectors
def to_one_hot_encoding(data_point_index):
    one_hot_encoding = np.zeros(ONE_HOT_DIM)
    one_hot_encoding[data_point_index] = 1
    return one_hot_encoding

X = [] # input word
Y = [] # target word

for x, y in zip(df['input'], df['label']):
    X.append(to_one_hot_encoding(word2int[ x ]))
    Y.append(to_one_hot_encoding(word2int[ y ]))

# convert them to numpy arrays
X_train = np.asarray(X)
Y_train = np.asarray(Y)

# making placeholders for X_train and Y_train
x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))

# word embedding will be 2 dimension for 2d visualization
EMBEDDING_DIM = 30

# hidden layer: which represents word vector eventually
W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([1])) #bias
hidden_layer = tf.add(tf.matmul(x,W1), b1)

# output layer
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM]))
b2 = tf.Variable(tf.random_normal([1]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2))

# loss function: cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), axis=[1]))

# training operation
train_op = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

In [31]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) 
saver = tf.train.Saver()

iteration = 30
#saver.restore(sess, "/tmp/model.ckpt")
for i in range(iteration):
    # input is X_train which is one hot encoded word
    # label is Y_train which is one hot encoded neighbor word
    sess.run(train_op, feed_dict={x: X_train, y_label: Y_train})
    if i % 10 == 0:
        print('iteration '+str(i)+' loss is : ', sess.run(loss, feed_dict={x: X_train, y_label: Y_train}))
        save_path = saver.save(sess, "/tmp/model.ckpt")
        print("Model saved in path: %s" % save_path)

iteration 0 loss is :  15.119047
Model saved in path: /tmp/model.ckpt
iteration 10 loss is :  14.535718
Model saved in path: /tmp/model.ckpt
iteration 20 loss is :  14.080547
Model saved in path: /tmp/model.ckpt


In [35]:
saver.restore(sess, "/tmp/model.ckpt")
iteration = 5001
for i in range(iteration):
    # input is X_train which is one hot encoded word
    # label is Y_train which is one hot encoded neighbor word
    sess.run(train_op, feed_dict={x: X_train, y_label: Y_train})
    if i % 1000 == 0:
        print('iteration '+str(i)+' loss is : ', sess.run(loss, feed_dict={x: X_train, y_label: Y_train}))
        save_path = saver.save(sess, "/tmp/model.ckpt")
        print("Model saved in path: %s" % save_path)

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
iteration 0 loss is :  2.0006204
Model saved in path: /tmp/model.ckpt
iteration 1000 loss is :  1.9844582
Model saved in path: /tmp/model.ckpt
iteration 2000 loss is :  1.9737483
Model saved in path: /tmp/model.ckpt
iteration 3000 loss is :  1.9663523
Model saved in path: /tmp/model.ckpt
iteration 4000 loss is :  1.960984
Model saved in path: /tmp/model.ckpt
iteration 5000 loss is :  1.9569153
Model saved in path: /tmp/model.ckpt


In [36]:
# Now the hidden layer (W1 + b1) is actually the word look up table
vectors = sess.run(W1 + b1)

In [54]:
w2v_df = pd.DataFrame(vectors, columns = ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30'])
w2v_df['word'] = words
w2v_df = w2v_df[['word', '1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30']]
w2v_df

,word,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,"에,JKB,*",-0.450492,1.201640,1.355451,-0.520889,-1.097278,-1.582678,-0.991922,-0.899409,1.103188,...,0.629952,0.626666,0.333584,-0.841012,0.837880,0.202465,0.141368,-0.416494,-0.605567,0.128693
1,"’,SY,*",1.268467,0.247846,-0.559522,0.703721,-0.823037,0.818977,-0.242935,0.268439,1.120607,...,-0.495274,1.127526,0.327250,1.278101,1.109099,0.786728,0.578589,-0.964117,-0.491642,0.082383
2,"축,NNBC,*",-1.267492,-0.285127,-0.272215,-0.486134,-0.947385,0.165914,0.349452,0.128904,1.387475,...,1.131226,0.230409,0.147620,0.576572,0.576265,0.601742,-0.229004,-0.781966,-0.193783,0.160557
3,"사용,NNG,행위",-0.035902,0.014542,1.075209,1.085015,-0.770309,0.047742,-0.122546,0.806388,-2.404690,...,-0.392870,-0.107673,1.143326,-0.214028,-1.240750,0.180255,0.545889,-0.615163,0.445102,-0.677784
4,"요건,NNG,*",-0.364116,-1.023491,0.176792,-1.125510,0.631321,0.101721,-0.207432,-0.756161,0.183430,...,0.971108,0.273239,-0.245639,0.225759,-0.239605,-0.343067,1.058850,-1.877740,-0.649904,-1.411389
5,"뇌물,NNG,*",-0.115747,-0.083375,0.125145,0.970829,0.363439,0.375086,0.822532,0.511021,-0.234953,...,-0.422072,1.597568,0.398335,0.927944,-0.589075,0.173840,-0.732503,0.208167,-0.293153,1.241942
6,"삭감,NNG,행위",1.966505,-0.655751,0.912012,-0.414552,-0.649266,0.586156,0.087993,-1.382488,1.979690,...,-1.239245,0.497155,0.879875,1.009436,0.892736,0.045347,-0.360051,0.089876,1.063268,-1.411924
7,"자료,NNG,*",-0.625614,-0.711740,0.771394,-0.392226,1.186219,-0.058320,-2.049412,-0.076107,-0.270698,...,0.145394,0.276219,-0.746424,1.354752,-0.567754,1.204574,-0.115140,0.029526,0.108036,-1.192463
8,"과,JKB,*",-1.305055,-0.876919,-0.943012,-0.208616,-0.351373,-0.521776,-1.233125,0.090607,0.905895,...,0.806972,-1.575452,-0.676258,-0.510461,0.862847,-0.125162,0.136024,0.020629,1.830496,-0.388868
9,"헌법,NNG,*",-0.611159,0.405336,2.007632,0.551538,-0.247582,0.039190,-0.544979,-2.281560,-0.343832,...,-1.080622,-0.459038,0.684650,-0.005113,-0.081767,-0.190489,0.333670,-1.791658,0.888173,-0.258570


In [95]:
word1 = "김정은,NNP,인명"
word2 = "문재인,NNP,인명"



#print(w2v_df.loc[w2v_df['word']== word1])

#print(w2v_df.loc[w2v_df['word']== word2])



# 한줄짜리 dataframe을 벡터로 바꾼다
import numpy as np
word1_values = (w2v_df.loc[w2v_df['word']== word1]).values
word1_values_no = np.delete(word1_values, 0, 1)

word2_values = (w2v_df.loc[w2v_df['word']== word2]).values
word2_values_no = np.delete(word2_values, 0, 1)


dist = np.linalg.norm(word1_values_no-word2_values_no)
print(dist)




def word_dist(word1, word2):
    word1_values = (w2v_df.loc[w2v_df['word']== word1]).values
    word1_values_no = np.delete(word1_values, 0, 1)
    word2_values = (w2v_df.loc[w2v_df['word']== word2]).values
    word2_values_no = np.delete(word2_values, 0, 1)
    dist = np.linalg.norm(word1_values_no-word2_values_no)
    return dist



7.818099738183198


In [102]:
print(word_dist("사용,NNG,행위", "국제,NNG,*"))

print(word_dist("文,NNG,*", "국제,NNG,*"))

print(word_dist("文,NNG,*", "퇴진,NNG,행위"))

print(word_dist("文,NNG,*", "문재인,NNP,인명"))

print(word_dist("김정은,NNP,인명", "문재인,NNP,인명"))

8.157348669774287
8.401562908739376
6.435440758117244
5.001404287031187
7.818099738183198
